In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')
import plotly as py
import plotly.graph_objs as go
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [2]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()
cf.set_config_file(offline=True, world_readable=True)

In [3]:
df = pd.read_excel('利率一级.xlsx',sheet_name = '数据（导入）')

In [4]:
# 这步是用来去除异常值的，之后有啥异常值还可以在这边改。
df = df[df['综收较估值'] < 40]

In [5]:
df_gk = df[df['发行人全称']=='国家开发银行'][['发行起始日','发行期限(年)','发行人全称','全场倍数','综收较估值','综收较二级']]
df_zj = df[(df['发行人全称'] == '国家开发银行')|(df['发行人全称'] == '中国农业发展银行')|(df['发行人全称'] == '中国进出口银行')][['发行起始日','发行期限(年)','发行人全称','全场倍数','综收较估值','综收较二级']]
maturity = [3,5,7,10]
df_gk

,发行起始日,发行期限(年),发行人全称,全场倍数,综收较估值,综收较二级
0,2015-01-06,7.0,国家开发银行,1.78,10.239519,NaN
1,2015-01-06,5.0,国家开发银行,2.68,-2.147878,NaN
2,2015-01-06,10.0,国家开发银行,1.78,7.776245,NaN
3,2015-01-13,7.0,国家开发银行,2.70,3.439959,NaN
4,2015-01-13,3.0,国家开发银行,5.04,-1.087258,NaN
...,...,...,...,...,...,...
3651,2021-04-29,3.0,国家开发银行,6.60,-2.112951,-2.552951
3652,2021-04-29,10.0,国家开发银行,3.21,-0.720000,-0.720000
3653,2021-05-06,1.0,国家开发银行,4.14,2.120601,-3.879399
3654,2021-05-06,5.0,国家开发银行,4.30,-2.291036,-1.041036


In [6]:
qcbs_quantile_gk = df_gk.groupby('发行期限(年)').apply(lambda df:np.nanquantile(df['全场倍数'],[0.25,0.5,0.75]))
zsjgz_quantile_gk = df_gk.groupby('发行期限(年)').apply(lambda df:np.nanquantile(df['综收较估值'],[0.25,0.5,0.75]))
qcbs_quantile_zj = df_zj.groupby('发行期限(年)').apply(lambda df:np.nanquantile(df['全场倍数'],[0.25,0.5,0.75]))
zsjgz_quantile_zj = df_zj.groupby('发行期限(年)').apply(lambda df:np.nanquantile(df['综收较估值'],[0.25,0.5,0.75]))



In [7]:
pyplt = py.offline.plot

for i in maturity:   
    temp = df_gk[df_gk['发行期限(年)']==i]
    trace1 = go.Scatter(
        x = temp['发行起始日'],
        y = temp['全场倍数'],
        fill = 'tozeroy',
        mode= 'none',# 无边界线
        name = "全场倍数",
        fillcolor = 'Lightblue'
    )
    trace2 = go.Scatter(
        x = temp['发行起始日'],
        y = temp['综收较估值'],
        mode= 'lines',# 无边界线
        name = "综收较估值",
        yaxis="y2",
        line = {'color': 'Chocolate'}
    )
#    trace3 =  go.Scatter(
#        x = temp['发行起始日'],
#        y = temp['综收较二级'],
#        mode= 'lines',# 无边界线
#        name = "综收较二级",
#        yaxis="y2",
#        line = {'color':'Cadetblue'}
#    )
    trace4 = go.Scatter(
        x = temp['发行起始日'],
        y = np.tile(qcbs_quantile_gk[qcbs_quantile_gk.index ==i].iloc[0][0],2000),
        mode= 'lines',
        name = "全场倍数25%",
        line={
        "width": 3,  
        "color": "Dodgerblue",  
        "dash": "dash"  # 指定为虚线
        })
    
    trace5 = go.Scatter(
        x = temp['发行起始日'],
        y = np.tile(qcbs_quantile_gk[qcbs_quantile_gk.index ==i].iloc[0][1],2000),
        mode= 'lines',
        name = "全场倍数50%",
        line={
        "width": 1,  
        "color": "Dodgerblue",  
        "dash": "dash"  # 指定为虚线
        }
    )
    trace6 =  go.Scatter(
        x = temp['发行起始日'],
        y = np.tile(qcbs_quantile_gk[qcbs_quantile_gk.index ==i].iloc[0][2],2000),
        mode= 'lines',
        name = "全场倍数75%",
        line={
        "width": 3,  
        "color": "Dodgerblue",  
        "dash": "dash"  # 指定为虚线
        }
    )
    trace7= go.Scatter(
        x = temp['发行起始日'],
        y = np.tile(zsjgz_quantile_gk[zsjgz_quantile_gk.index ==i].iloc[0][0],2000),
        mode= 'lines',
        name = "综收较估值25%",
        line={
        "width": 3,  
        "color": "Brown",  
        "dash": "dash"  # 指定为虚线
        },
        yaxis="y2"
    )
    trace8 = go.Scatter(
        x = temp['发行起始日'],
        y = np.tile(zsjgz_quantile_gk[zsjgz_quantile_gk.index ==i].iloc[0][1],2000),
        mode= 'lines',
        name = "综收较估值50%",
        yaxis="y2",
        line={
        "width": 1,  
        "color": "Brown",  
        "dash": "dash"  # 指定为虚线
        },
    )
    trace9 =  go.Scatter(
        x = temp['发行起始日'],
        y = np.tile(zsjgz_quantile_gk[zsjgz_quantile_gk.index ==i].iloc[0][2],2000),
        mode= 'lines',
        name = "综收较估值75%",
        yaxis="y2",
        line={
        "width": 3,  
        "color": "Brown",  
        "dash": "dash"  # 指定为虚线
        }
    )   
    data = [trace1, trace2,trace4,trace5,trace6,trace7,trace8,trace9]
    layout = go.Layout(title =dict(text = ''.join(['国开',str(i),'年']),
                                    x = 0.5,
                                    yanchor = 'middle') ,
                      yaxis = dict(title = '全场倍数'),
                      yaxis2 = dict(title = '综收较估值',overlaying='y', side='right'),
                       xaxis = dict( 
                      tickformat = '%Y-%m', 
                      tick0 = '2015-01-01',
                      dtick = 'M3',
                     ticks = 'inside'), # 设置每隔一季度就是一个tick
                       legend=dict(orientation="h"),
              #         height = 800,
              #        width = 1000)
                      )
                       
                       
    
                    
        
    #设置图离图像四周的边距
    
    fig = go.Figure(data = data, layout = layout)#设置图像的大小
    pyplt(fig, filename = ''.join(['国开',str(i),'年']))
#    pyplt(fig, filename = ''.join(['国开债',str(i),'年']))


'国开3年.html'

'国开5年.html'

'国开7年.html'

'国开10年.html'

In [8]:
for i in maturity:   
    temp = df_zj[df_zj['发行期限(年)']==i]    
    trace1 = go.Scatter(
        x = temp.groupby('发行起始日').apply(lambda df: df['全场倍数'].mean()).sort_index().index,
        y = temp.groupby('发行起始日').apply(lambda df: df['全场倍数'].mean()).sort_index(),
        fill = 'tozeroy',
        mode= 'none',# 无边界线
        name = "全场倍数",
        fillcolor = 'Lightblue'
    )
    trace2 = go.Scatter(
        x = temp.groupby('发行起始日').apply(lambda df: df['综收较估值'].mean()).sort_index().index,
        y = temp.groupby('发行起始日').apply(lambda df: df['综收较估值'].mean()).sort_index(),
        mode= 'lines',# 无边界线
        name = "综收较估值",
        yaxis="y2",
        line = {'color': 'Chocolate'}
    )
#    trace3 =  go.Scatter(
#        x = temp['发行起始日'],
#        y = temp['综收较二级'],
#        mode= 'lines',# 无边界线
#        name = "综收较二级",
#        yaxis="y2",
#        line = {'color':'Cadetblue'}
#    )
    trace4 = go.Scatter(
        x = temp['发行起始日'],
        y = np.tile(qcbs_quantile_zj[qcbs_quantile_zj.index ==i].iloc[0][0],2000),
        mode= 'lines',
        name = "全场倍数25%",
        line={
        "width": 3,  
        "color": "Dodgerblue",  
        "dash": "dash"  # 指定为虚线
        })
    
    trace5 = go.Scatter(
        x = temp['发行起始日'],
        y = np.tile(qcbs_quantile_zj[qcbs_quantile_zj.index ==i].iloc[0][1],2000),
        mode= 'lines',
        name = "全场倍数50%",
        line={
        "width": 1,  
        "color": "Dodgerblue",  
        "dash": "dash"  # 指定为虚线
        }
    )
    trace6 =  go.Scatter(
        x = temp['发行起始日'],
        y = np.tile(qcbs_quantile_zj[qcbs_quantile_zj.index ==i].iloc[0][2],2000),
        mode= 'lines',
        name = "全场倍数75%",
        line={
        "width": 3,  
        "color": "Dodgerblue",  
        "dash": "dash"  # 指定为虚线
        }
    )
    trace7= go.Scatter(
        x = temp['发行起始日'],
        y = np.tile(zsjgz_quantile_zj[zsjgz_quantile_zj.index ==i].iloc[0][0],2000),
        mode= 'lines',
        name = "综收较估值25%",
        line={
        "width": 3,  
        "color": "Brown",  
        "dash": "dash"  # 指定为虚线
        },
        yaxis="y2"
    )
    trace8 = go.Scatter(
        x = temp['发行起始日'],
        y = np.tile(zsjgz_quantile_zj[zsjgz_quantile_zj.index ==i].iloc[0][1],2000),
        mode= 'lines',
        name = "综收较估值50%",
        yaxis="y2",
        line={
        "width": 1,  
        "color": "Brown",  
        "dash": "dash"  # 指定为虚线
        },
    )
    trace9 =  go.Scatter(
        x = temp['发行起始日'],
        y = np.tile(zsjgz_quantile_zj[zsjgz_quantile_zj.index ==i].iloc[0][2],2000),
        mode= 'lines',
        name = "综收较估值75%",
        yaxis="y2",
        line={
        "width": 3,  
        "color": "Brown",  
        "dash": "dash"  # 指定为虚线
        }
    )   
    data = [trace1, trace2,trace4,trace5,trace6,trace7,trace8,trace9]
    layout = go.Layout(title =dict(text = ''.join(['政金',str(i),'年']),
                                    x = 0.5,
                                    yanchor = 'middle') ,
                      yaxis = dict(title = '全场倍数'),
                      yaxis2 = dict(title = '综收较估值',overlaying='y', side='right'),
                       xaxis = dict( 
                      tickformat = '%Y-%m', 
                      tick0 = '2015-01-01',
                      dtick = 'M3',
                     ticks = 'inside'), # 设置每隔一季度就是一个tick
                       legend=dict(orientation="h"),
                  #     height = 750)
                      )
                       
                       
    
                    
        
    #设置图离图像四周的边距
    
    fig = go.Figure(data = data, layout = layout)#设置图像的大小
    pyplt(fig, filename = ''.join(['政金',str(i),'年']))


'政金3年.html'

'政金5年.html'

'政金7年.html'

'政金10年.html'

In [9]:
df_2020 = df[df['发行起始日']>'2019-12-31'][['发行起始日','发行期限(年)','发行人全称','全场倍数','综收较估值','综收较二级']]
maturity = [3,5,7,10]
df_qcbs_quantile_2020 = df.groupby(['发行人全称','发行期限(年)']).apply(lambda df:np.nanquantile(df['全场倍数'],[0.25,0.5,0.75])).reset_index()
df_zsjgz_quantile_2020 = df.groupby(['发行人全称','发行期限(年)']).apply(lambda df:np.nanquantile(df['综收较估值'],[0.25,0.5,0.75])).reset_index()


In [10]:
df_2020.head()

,发行起始日,发行期限(年),发行人全称,全场倍数,综收较估值,综收较二级
2360,2020-01-07,1.0,国家开发银行,2.72,-8.940578,-8.780578
2361,2020-01-16,1.0,国家开发银行,2.50,1.839750,1.009750
2362,2020-01-21,1.0,国家开发银行,4.18,0.764572,-1.255428
2363,2020-02-06,1.0,国家开发银行,11.85,-35.580000,-35.000000
2364,2020-02-13,1.0,国家开发银行,4.06,-8.387746,-7.457746


In [11]:
maturity2020 = [1,3,5,7,10]

In [12]:
issuer = df['发行人全称'].unique().tolist()
issuer

['国家开发银行', '中华人民共和国财政部', '中国农业发展银行', '中国进出口银行']

In [13]:
issuer = df['发行人全称'].unique().tolist()
for i in issuer:
    for year in maturity2020:
        temp = df_2020[(df_2020['发行人全称']==i)&(df_2020['发行期限(年)']==year)]    
        trace1 = go.Scatter(
            x = temp.groupby('发行起始日').apply(lambda df: df['全场倍数'].mean()).sort_index().index,
            y = temp.groupby('发行起始日').apply(lambda df: df['全场倍数'].mean()).sort_index(),
            fill = 'tozeroy',
            mode= 'none',# 无边界线
            name = "全场倍数",
            fillcolor = 'Lightblue'
        )
        trace2 = go.Scatter(
            x = temp.groupby('发行起始日').apply(lambda df: df['综收较估值'].mean()).sort_index().index,
            y = temp.groupby('发行起始日').apply(lambda df: df['综收较估值'].mean()).sort_index(),
            mode= 'lines',# 无边界线
            name = "综收较估值",
            yaxis="y2",
            line = {'color': 'Chocolate'}
        )
        trace3 =  go.Scatter(
            x = temp.groupby('发行起始日').apply(lambda df: df['综收较二级'].mean()).sort_index().index,
            y = temp.groupby('发行起始日').apply(lambda df: df['综收较二级'].mean()).sort_index(),
            mode= 'lines',# 无边界线
            name = "综收较二级",
            yaxis="y2",
            line = {'color':'Firebrick'}
        )
        trace4 = go.Scatter(
            x = temp['发行起始日'],
            y = np.tile(df_qcbs_quantile_2020[(df_qcbs_quantile_2020['发行人全称'] == i)&(df_qcbs_quantile_2020['发行期限(年)'] == year)].iloc[:,2].tolist()[0][0],200),
            mode= 'lines',
            name = "全场倍数25%",
            line={
            "width": 3,  
            "color": "Dodgerblue",  
            "dash": "dash"  # 指定为虚线
            })

        trace5 = go.Scatter(
            x = temp['发行起始日'],
            y = np.tile(df_qcbs_quantile_2020[(df_qcbs_quantile_2020['发行人全称'] == i)&(df_qcbs_quantile_2020['发行期限(年)'] == year)].iloc[:,2].tolist()[0][1],200),
            mode= 'lines',
            name = "全场倍数50%",
            line={
            "width": 1,  
            "color": "Dodgerblue",  
            "dash": "dash"  # 指定为虚线
            }
        )
        trace6 =  go.Scatter(
            x = temp['发行起始日'],
            y = np.tile(df_qcbs_quantile_2020[(df_qcbs_quantile_2020['发行人全称'] == i)&(df_qcbs_quantile_2020['发行期限(年)'] == year)].iloc[:,2].tolist()[0][2],200),
            mode= 'lines',
            name = "全场倍数75%",
            line={
            "width": 3,  
            "color": "Dodgerblue",  
            "dash": "dash"  # 指定为虚线
            }
        )
        trace7= go.Scatter(
            x = temp['发行起始日'],
            y = np.tile(df_zsjgz_quantile_2020[(df_zsjgz_quantile_2020['发行人全称'] == i)&(df_zsjgz_quantile_2020['发行期限(年)'] == year)].iloc[:,2].tolist()[0][0],200),
            mode= 'lines',
            name = "综收较估值25%",
            line={
            "width": 3,  
            "color": "Brown",  
            "dash": "dash"  # 指定为虚线
            },
            yaxis="y2"
        )
        trace8 = go.Scatter(
            x = temp['发行起始日'],
            y = np.tile(df_zsjgz_quantile_2020[(df_zsjgz_quantile_2020['发行人全称'] == i)&(df_zsjgz_quantile_2020['发行期限(年)'] == year)].iloc[:,2].tolist()[0][1],200),
            mode= 'lines',
            name = "综收较估值50%",
            yaxis="y2",
            line={
            "width": 1,  
            "color": "Brown",  
            "dash": "dash"  # 指定为虚线
            },
        )
        trace9 =  go.Scatter(
            x = temp['发行起始日'],
            y = np.tile(df_zsjgz_quantile_2020[(df_zsjgz_quantile_2020['发行人全称'] == i)&(df_zsjgz_quantile_2020['发行期限(年)'] == year)].iloc[:,2].tolist()[0][2],200),
            mode= 'lines',
            name = "综收较估值75%",
            yaxis="y2",
            line={
            "width": 3,  
            "color": "Brown",  
            "dash": "dash"  # 指定为虚线
            }
        )   
        data = [trace1, trace2,trace3,trace4,trace5,trace6,trace7,trace8,trace9]
        name = '国开'
        if i == '中华人民共和国财政部':
            name = '国债'
        elif i == '中国农业发展银行':
            name = '农发'
        elif i == '中国进出口银行':
            name = '口行'
        
        layout = go.Layout(title =dict(text = ''.join([name,str(year),'年']),
                                        x = 0.5,
                                        yanchor = 'middle') ,
                          yaxis = dict(title = '全场倍数'),
                          yaxis2 = dict(title = '综收较估值',overlaying='y', side='right'),
                           xaxis = dict( 
                           type = 'date',
                            tickformat = '%Y-%m-%d',
                          tick0 = '2020-01-01',
                          dtick = 86400000*15,
                         ticks = 'inside'), # 设置每隔一季度就是一个tick
                           legend=dict(orientation="h")
                     #      height = 750,
                    #      width = 1400)
                          )




        #设置图离图像四周的边距

        fig = go.Figure(data = data, layout = layout)#设置图像的大小
        pyplt(fig, filename = ''.join([i,str(year),'年']))


'国家开发银行1年.html'

'国家开发银行3年.html'

'国家开发银行5年.html'

'国家开发银行7年.html'

'国家开发银行10年.html'

...

'中国进出口银行1年.html'

'中国进出口银行3年.html'

'中国进出口银行5年.html'

'中国进出口银行7年.html'

'中国进出口银行10年.html'